# Set the environment

In [1]:
# set the envoiroment
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import escher
from escher import Builder
from utils import show_map
from utils.check_precursor_problem import check_precursor_problem
map_loc = "../data/fermentation.json"
map_loc1 = "../data/figureS4E1.json"

## Read the model

In [2]:
model = cobra.io.load_json_model("../models/updated_model.json")# Import the model
model.solver = 'glpk'
model.reactions.get_by_id("prot_prot__lactate").bounds = (-1000,1000)
model.reactions.get_by_id("NGAM").bounds = (0,1000)
model.reactions.T_flux_oxygen.bounds = (0,0) # block the import of oxygen

In [3]:
### Pumping 0 protons from lactic acid export system 
Psi = Metabolite(
    'Psi',
    formula='',
    name='PSI is a unit of pressure expressed in pounds of force per square inch of area',
    compartment='cytosol')
model.reactions.get_by_id('T_symporter_(S)__lactic_acid').remove_from_model()
reaction = Reaction('T_symporter_(S)__lactic_acid')
reaction.name = 'T_symporter_(S)__lactic_acid'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.add_metabolites({"(S)__lactic_acid":-1.0,
                         "H+":-0.0,
                         "(S)__lactic_acid_ex":1.0,
                           Psi: 0.0,
                          "H+_ex":0.0})

In [4]:
model.objective = model.reactions.Biomass
sol = model.optimize()
print (sol)

<Solution 0.000 at 0x1ba13a0e550>


## ATP source reaction was added

In [5]:
## Use the lactic acid export reaction as objective reaction
model1 = model.copy()
reaction = Reaction('ATP_source_reaction')
reaction.name = 'ATP_source_reaction'
reaction.lower_bound = -0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model1.add_reactions([reaction])
reaction.add_metabolites({"ADP": -1.0,
                          "H+": -1.0,
                          "phosphate":-1.0,
                          "H2O": 1.0,
                         "ATP": 1.0})
model1.objective = model1.reactions.get_by_id('T_flux_(S)__lactic_acid')

# checking different n values for the ATP source reaction


## n =-1,solver status is 'infeasible'

In [6]:
### Use the lactate dehydrogenase reaction as objective reaction
M= model1.copy()
M.reactions.ATP_source_reaction.bounds= (-1,-1)
sol = M.optimize()
print (sol)

<Solution infeasible at 0x1ba1626b0b8>


cobra\util\solver.py:408 UserWarning: solver status is 'infeasible'


## n = 0,PGK and GAPDH pathway (Fig.S1A)

In [7]:
M= model1.copy()
M.reactions.ATP_source_reaction.bounds= (-0,-0)
sol = M.optimize()
b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

# Blocking PGK

## n = 0, solver status is 'infeasible'

In [8]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (-0,-0)
sol = M.optimize()
print (sol)

<Solution 0.000 at 0x1ba17bcbeb8>


## n = 1

### spontaneous degradation pathway (Fig.S1B) 

In [9]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (1,1)
sol = M.optimize()
b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### Blocking T_ABC transport_glucose and PGK

In [10]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('T_ABC__transporter_beta__D__glucose').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (1,1)
sol = M.optimize()
print (sol)

<Solution 0.000 at 0x1ba181c1588>


### Non-phosphorylation pathway (Fig.S1C) after blocking the reactions for KDGK and PGK
    - There are two reactions for KDGK enzyme, after making the bounds to 0, the 
    non-phosphorylation pathway will be used. 
    
    - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GK reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [11]:
M= model1.copy()
# The following two reactions are the ones for KDGK reactions
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (1,1)
sol = M.optimize()
b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### GAPN pathway (Fig.S1D) after blocking GKI, spondegradation for BPG.
      - In the figure we can see that GAPDH reactions will also be used, the reason for this is the NADPH produced in the GAPN reaction need to be changed to NADH, and these reactions are used for recycling of NADPH. 

In [12]:
M= model1.copy()
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (1,1)
sol = M.optimize()
b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

# Blocking PK reactions

## n = -1, solver status is 'infeasible'

In [13]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (-1,-1)
sol = M.optimize()
print (sol)

<Solution infeasible at 0x1ba18aa7e48>


## n = 0, solver status is 'infeasible'

In [14]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (-0,-0)
sol = M.optimize()
print (sol)

<Solution 0.000 at 0x1ba19f37908>


## n = 1

### Fig.S1E

In [15]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (1,1)
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### Fig. S1E1

In [16]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_anap_4.1.1.31_PEPCARBOX__RXN').bounds = (0,0)
# M.reactions.get_by_id("aa_met_4.4.1.8_CYSTATHIONINE__BETA__LYASE__RXN").bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (1,1)
sol = M.optimize()

b = show_map(sol,map_loc1)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### Adding serine dehydratase reaction into the model

In [17]:
reaction = Reaction('aa_ser_gly_thr_4.3.1.19_L_SERINE_AMMONIA_LYASE_RXN')
reaction.name = 'aa_ser_gly_thr_4.3.1.19_L_SERINE_AMMONIA_LYASE_RXN'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model1.add_reactions([reaction])
reaction.add_metabolites({"L__serine":-1.0,
                         "pyruvate":1.0,
                         "ammonia": 1.0})
model1.reactions.get_by_id('aa_met_salvage_4.2.1.22_CYSTATHIONINE__BETA__SYNTHASE__RXN').bounds = (0,0)# these two reactions do not exist in this organism

### Fig.S1F

In [18]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_anap_4.1.1.31_PEPCARBOX__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (1,1)
sol = M.optimize()

b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## Blocking both PGK and PK reactions

### n = 0, solver status is "infeasible"

In [19]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (0,0)
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print (sol)

<Solution 0.000 at 0x1ba1ab298d0>


### n = 1, solver status is "infeasible"

In [20]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (1,1)
M.reactions.get_by_id("T_flux_beta__D__glucose").bounds = (-1,-1)
sol = M.optimize()
print (sol)

<Solution infeasible at 0x1ba1afe7358>


### n = 2

### Fig.S1G

In [21]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (2,2)
sol = M.optimize()

b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### Fig.S1H

In [22]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_anap_4.1.1.31_PEPCARBOX__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (2,2)
sol = M.optimize()

b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### blocking KDGK

#### Fig.S1I, blocking KDPG, using non phosphorylation pathway

In [23]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('aa_ser_gly_cys_sec_trp_1.1.1.95_PGLYCDEHYDROG__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (2,2)
sol = M.optimize()
b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

#### Fig. S1J, blocking KDGK, using spontaneous pathway

In [24]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEHYDDEOXGALACTKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.178_DEOXYGLUCONOKIN__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_anap_4.1.1.31_PEPCARBOX__RXN').bounds = (0,0)

M.reactions.ATP_source_reaction.bounds= (2,2)
sol = M.optimize()

b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### Blocking GKI

#### Fig.S1H Blocking GKI,  use spontaneous degradation pathway

In [25]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_anap_4.1.1.31_PEPCARBOX__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (2,2)
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
# for rxn in l:
#     print (rxn,M.reactions.get_by_id(rxn).reaction,sol.fluxes.loc[rxn])
b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### blocking GKI and spontaneous

#### Fig.S1K

In [30]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)

M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (-2,2)
sol = M.optimize()

b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

#### Fig.S1L

In [31]:
M= model1.copy()
M.reactions.get_by_id('carb_enter_2.7.1.40_PEPDEPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_enter_glucneo_2.7.2.3_PHOSGLYPHOS__RXN').bounds = (0,0)
M.reactions.get_by_id('carb_entner_2.7.1.165_GKI__RXN').bounds = (0,0)
M.reactions.get_by_id('other_no__pathway_3.6.1.7_BS165362').bounds = (0,0)

M.reactions.get_by_id('aa_ser_gly_cys_sec_trp_1.1.1.95_PGLYCDEHYDROG__RXN').bounds = (0,0)
M.reactions.ATP_source_reaction.bounds= (-2,2)
sol = M.optimize()

b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…